# 3.3 Discontinuous Galerkin Discretizations for linear transport
We are solving the scalar linear transport problem

Find $u: [0,T] \to V_D := \{ u \in L^2(\Omega), b \cdot \nabla u \in L^2(\Omega), u|_{\Gamma_{in}} = u_D\}$, s.t.
\begin{equation}
\int_{\Omega} \partial_t u v +  b \cdot \nabla u v = \int_{\Omega} f v \qquad \forall v \in V_0 = \{ u \in L^2(\Omega), b \cdot \nabla u \in L^2(\Omega), u|_{\Gamma_{in}} = 0\}
\end{equation}

In [ ]:
from netgen import gui
from math import pi
from ngsolve import *

As a first example, we consider the unit square $(0,1)^2$ and the advection velocity $b = (1,2)$. Accordingly the inflow boundary is $\Gamma_{in} = \{ x \cdot y = 0\}$. 

In [ ]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (1, 1), 
                 bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.2))

We consider an Upwind DG discretization (in space):

Find $u: [0,T] \to V_h := \bigoplus_{T\in\mathcal{T}_h} \mathcal{P}^k(T)$ so that

$$
  \sum_{T} \int_T \partial_t u v + b \cdot \nabla u v + \int_{\partial T} b_n (\hat{u}-u) v = \int_{\Omega} f v, \quad \forall v \in V_h.
$$

Here $\hat{u}$ is the Upwind flux, i.e. $\hat{u} = u$ on the outflow boundary part $\partial T_{out} = \{ x\in \partial T \mid b(x) \cdot n_T(x) \geq 0 \}$ of $T$ and $\hat{u} = u^{other}$ else, with $u^{other}$ the value from the neighboring element.

There is quite a difference in the computational costs (compared to a standard DG formulation) depending on the question if the solution of linear systems is involved or only operator evaluations (explicit method). 

 * We treat the explicit case here separately 
 * and refer to [unit 2.8](../unit-2.8-DG/DG.ipynb) for solving linear systems with DG formulations.

## Explicit time stepping with a DG formulation

Explicit Euler:
$$
\sum_{T} \int_T u^{n+1} v = \sum_{T} \int_T u^{n} v - \Delta t \sum_{T} \left\{ \int_T  b \cdot \nabla u v 
+ \int_{\partial T} b_n (\hat{u}-u) v \right\} - \Delta t \int_{\Omega} f v, \quad \forall v \in V_h,
$$
$$
  M u^{n+1} = M u^{n} - \Delta t C u^n + \Delta t f
$$

In our first example we set $u_0 = f = 0$.


### Computing convection applications $C u^n$
* We can define the bilinear form **without setting up a matrix** with storage. (`nonassemble=True`)
* A `BilinearForm` is allowed to be **nonlinear in the 1st argument** (for operator applications)
* For the DG formulation we require integrals on element boundaries, keyword: **`element_boundary`**
* To distinguish inflow from outflow we evaluate $b \cdot \mathbf{n}$. Here the normal $\mathbf{n}$ is available as a `specialcf`
* To make cases with `CoefficientFunctions` we use the `IfPos`-`CoefficientFunction`. **`IfPos(a,b,c)`**. `a` decides on the evaluation. If `a` is positive `b` is evaluated, otherwise `c`.
* To access the **neighbor** function we can use `u.Other()`
* To incorporate **boundary conditions** ($\hat{u}$ on inflow boundaries), we can use the argument `bnd` of `.Other(bnd)`. If there is no neighbor element (boundary!) the `CoefficientFunction` `bnd` is evaluated. 

In [ ]:
b = CoefficientFunction((1,2))
n = specialcf.normal(mesh.dim)
ubnd = IfPos(x,1,0)

V = L2(mesh,order=2)
u,v = V.TrialFunction(), V.TestFunction()

c = BilinearForm(V, nonassemble=True)
c += b * grad(u) * v * dx
c += IfPos( (b*n), 0, (b*n) * (u.Other(ubnd)-u)) * v * dx(element_boundary=True)

gfu_expl = GridFunction(V)
Draw(gfu_expl,mesh,"u_explicit")

# Bilinearform for operator applications
Due to the `nonassemble=True` in the constructor of the `BilinearForm` we can use `c.mat * x` to realize the operator application with assembling a matrix first:

In [ ]:
res = gfu_expl.vec.CreateVector()
#Operator application (equivalent to assemble and mult but faster)
res.data = c.mat * gfu_expl.vec 

### Solving mass matrix problems

* Need to invert the mass matrix
* For DG methods the mass matrix is block diagonal, often even diagonal.
* FESpace offers (if available):
  * `Mass(rho)`: mass matrix as an operator  (not a sparse matrix)
  * `Mass(rho).Inverse()`: inverse mass matrix as an operator (not a sparse matrix)

In [ ]:
invm = V.Mass(1).Inverse()

In [ ]:
t = 0
dt = 0.001
tend = 1

while t < tend-0.5*dt:
    res.data = invm @ c.mat * gfu_expl.vec     
    gfu_expl.vec.data -= dt * res
    t += dt
    Redraw(blocking=True)

## Supplementary: Skeleton formulation

So far we considered the DG formulation with integrals on the boundary of each element. Instead one could formulate the problem in terms of facet integrals where every facet appears only once. 

Then, the corresponding formulation is:

Find $u: [0,T] \to V_h := \bigoplus_{T\in\mathcal{T}_h} \mathcal{P}^k(T)$ so that
$$
  \sum_{T} \int_T \partial_t u v + b \cdot \nabla u v + \sum_{F\in\mathcal{F}^{int}} \int_{F} b_n (u^{neighbor} - u) v^{downwind} + \\
  \sum_{F\in\mathcal{F}^{inflow}} \int_{F} b_n (u^{inflow}-u) v = \int_{\Omega} f v , \quad \forall v \in V_h.
$$

Here $\mathcal{F}^{int}$ is the set of interior facets and $\mathcal{F}^{inflow}$ is the set of boundary facets where $b \cdot \mathbf{n} < 0$. $v^{downwind}$ is the function on the downwind side of the facet and $u^{inflow}$ is the inflow boundary condition. 

The facet integrals are divided into interior facets and exterior (boundary) facets.
To obtain these integrals we combine the `DifferentialSymbol` `dx` or `ds` with `skeleton=True`.

In [ ]:
dskel_inner  = dx(skeleton=True)
dskel_bound  = ds(skeleton=True)
# or:
dskel_inflow = ds(skeleton=True, definedon=mesh.Boundaries("left|bottom"))

In [ ]:
b = CoefficientFunction((1,2))
n = specialcf.normal(mesh.dim)
ubnd = IfPos(x,1,0)

V = L2(mesh,order=2)
u,v = V.TrialFunction(), V.TestFunction()

c = BilinearForm(V)
c += b * grad(u) * v * dx


bn = b*n
vin = IfPos(bn,v.Other(),v)
c += bn*(u.Other() - u) * vin * dskel_inner
#c += IfPos(bn, 0, bn) * (u.Other(ubnd) - u) * v * dskel_bound
#alternatively:
c += bn * (u.Other(ubnd) - u) * v * dskel_inflow

gfu_expl = GridFunction(V)
Draw(gfu_expl,mesh,"u_explicit")

res = gfu_expl.vec.CreateVector()
c.Apply(gfu_expl.vec,res)

In [ ]:
t = 0
dt = 0.001
tend = 1

while t < tend-0.5*dt:
    c.Apply(gfu_expl.vec,res)
    V.SolveM(res,rho=CoefficientFunction(1.0))
    gfu_expl.vec.data -= dt * res
    t += dt
    Redraw(blocking=True)

DG for Diffusion has been introduced in [unit 2.8 (Hybrid) Discontinuous Galerkin methods](../unit-2.8-DG/DG.ipynb).